In [ ]:
#testing google colab commit

In [ ]:
!pip3 install duckduckgo-search newspaper4k lxml_html_clean

# !pip3 install --upgrade --quiet duckduckgo-search
# use this if rate limit is reached

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 9.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=92077276ec351f574b5605b3e4d70f57aa6a3feb6409bbf73f9a4116df084646
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [ ]:
import requests
import json

target_summaries_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/target.json"

# Fetch the JSON data from the URL
response = requests.get(target_summaries_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
target_summaries = json.loads(response.text)

# Now you can access the data within the target_summaries dictionary
print(target_summaries)  # Print the loaded JSON data

In [ ]:
import json
import time
from duckduckgo_search import DDGS
import requests
import os

def get_news_for_query(query, max_results=10):
    """Fetch news results for a single query"""
    try:
        ddgs = DDGS()
        results = list(ddgs.news(
            keywords=query,
            region="us-en",
            safesearch="off",
            timelimit="m",
            max_results=max_results
        ))

        # Extract just the URLs
        urls = [item['url'] for item in results]
        return urls

    except Exception as e:
        print(f"Error searching for '{query}': {str(e)}")
        return []

def main():
    # Detect if running in Colab
    in_colab = 'google.colab' in str(get_ipython())

    # Fetch the target summaries
    target_summaries_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/target.json"
    response = requests.get(target_summaries_url)
    response.raise_for_status()
    target_summaries = json.loads(response.text)

    # Dictionary to store query -> URLs mapping
    query_urls = {}

    # Process each query in the target summaries
    for item in target_summaries:
        query = item['query']
        print(f"\nProcessing query: {query}")

        # Get URLs for this query
        urls = get_news_for_query(query, max_results=10)
        query_urls[query] = urls

        print(f"Found {len(urls)} URLs for query: {query}")

        # Wait between different queries to avoid rate limiting
        if target_summaries.index(item) < len(target_summaries) - 1:  # Don't wait after the last query
            delay = 30  # 30 seconds between queries
            print(f"Waiting {delay} seconds before the next query...")
            time.sleep(delay)

    # Save the collected URLs to a JSON file
    output_file = 'news_urls.json'
    with open(output_file, 'w') as f:
        json.dump(query_urls, f, indent=2)

    print(f"\nAll URLs collected and saved to {output_file}")

    # If in Colab, provide code to download the file
    if in_colab:
        from google.colab import files
        files.download(output_file)
        print("\nFile download initiated. Check your browser's downloads.")

        # Also display the content for convenience
        print("\nJSON content:")
        print(json.dumps(query_urls, indent=2))

if __name__ == "__main__":
    main()

In [ ]:
import newspaper

query_urls = {}

# Load the URLs from the JSON file
with open('news_urls.json', 'r') as f:
    query_urls = json.load(f)

30


In [ ]:
print(len(query_urls))
print(query_urls)

In [ ]:
import json
import time
import newspaper
import requests
from newspaper import Article
import os

def fetch_article_text(url):
    """Extract full text from a given news URL using newspaper"""
    try:
        article = Article(url)
        article.download()
        article.parse()

        return {
            'title': article.title,
            'text': article.text,
            'url': url
        }
    except Exception as e:
        print(f"Error fetching article from {url}: {str(e)}")
        return {
            'title': '',
            'text': '',
            'url': url,
            'error': str(e)
        }

def main():
    # Detect if running in Colab
    try:
        from google.colab import files
        in_colab = True
    except ImportError:
        in_colab = False

    # Load the URLs file
    try:
        with open('news_urls.json', 'r') as f:
            news_urls = json.load(f)
    except FileNotFoundError:
        print("news_urls.json not found. Please run the URL collection script first.")
        return

    # Dictionary to store all article data
    article_data = {}

    # Process each query
    for i, (query, urls) in enumerate(news_urls.items()):
        print(f"\nProcessing query {i+1}/{len(news_urls)}: {query}")
        print(f"Found {len(urls)} URLs to process")

        # Initialize array for this query's articles
        article_data[query] = []

        # Process each URL for this query
        for j, url in enumerate(urls):
            print(f"  Processing URL {j+1}/{len(urls)}: {url}")
            article_info = fetch_article_text(url)

            # Add to our data structure
            article_data[query].append(article_info)

            # Brief pause to be gentle on servers
            if j < len(urls) - 1:
                time.sleep(1)

        # Add a longer pause between queries
        if i < len(news_urls) - 1:
            print("Waiting 5 seconds before next query...")
            time.sleep(5)

    # Save to a new JSON file
    output_file = 'article_texts.json'
    with open(output_file, 'w') as f:
        json.dump(article_data, f, indent=2)

    print(f"\nAll article texts collected and saved to {output_file}")

    # If in Colab, provide download option
    if in_colab:
        files.download(output_file)
        print("\nFile download initiated. Check your browser's downloads.")

        # Display summary statistics
        total_articles = sum(len(articles) for articles in article_data.values())
        successful_articles = sum(1 for query_articles in article_data.values()
                                for article in query_articles if article['text'])

        print(f"\nSummary statistics:")
        print(f"Total queries processed: {len(article_data)}")
        print(f"Total articles attempted: {total_articles}")
        print(f"Successfully parsed articles: {successful_articles} ({successful_articles/total_articles*100:.1f}%)")

if __name__ == "__main__":
    main()


Processing query 1/30: Global economic outlook
Found 10 URLs to process
  Processing URL 1/10: https://www.msn.com/en-ca/money/topstories/global-economic-growth-could-slow-to-23-due-to-trade-tensions-says-un-trade-agency/ar-AA1D1z6Q
  Processing URL 2/10: https://seekingalpha.com/article/4775157-how-tariffs-geopolitics-shaping-2025-global-economic-outlook


Error fetching article from https://seekingalpha.com/article/4775157-how-tariffs-geopolitics-shaping-2025-global-economic-outlook: Article `download()` failed with Status code 403 for url None on URL https://seekingalpha.com/article/4775157-how-tariffs-geopolitics-shaping-2025-global-economic-outlook
  Processing URL 3/10: https://www.cnet.com/personal-finance/refinance-rates-tick-higher-mortgage-refinance-rates-on-april-11-2025/
  Processing URL 4/10: https://www.businessinsider.com/us-economy-stock-market-outlook-tariffs-trump-trade-war-bofa-2025-4
  Processing URL 5/10: https://www.msn.com/en-us/money/markets/economic-outlook-dives-just-three-months-into-trump-s-term/ar-AA1COpd8
  Processing URL 6/10: https://www.msn.com/en-us/money/markets/u-k-economy-shows-signs-of-strength-but-tariffs-darken-outlook/ar-AA1CIv5r
  Processing URL 7/10: https://www.marketwatch.com/livecoverage/stock-market-today-dow-s-p-and-nasdaq-eye-steady-start-as-tariff-angst-eases-bofa-and-citi-earnings/card/ie

Error fetching article from https://www.marketwatch.com/livecoverage/stock-market-today-dow-s-p-and-nasdaq-eye-steady-start-as-tariff-angst-eases-bofa-and-citi-earnings/card/iea-cuts-2025-global-demand-growth-forecast-on-deteriorating-outlook-for-the-global-economy-BF64s9E3Q4RnPFCzj2xA: Article `download()` failed with Status code 401 for url None on URL https://www.marketwatch.com/livecoverage/stock-market-today-dow-s-p-and-nasdaq-eye-steady-start-as-tariff-angst-eases-bofa-and-citi-earnings/card/iea-cuts-2025-global-demand-growth-forecast-on-deteriorating-outlook-for-the-global-economy-BF64s9E3Q4RnPFCzj2xA
  Processing URL 8/10: https://www.msn.com/en-ca/money/other/fed-chair-powell-to-deliver-fresh-economic-view-as-tariffs-inject-uncertainty/ar-AA1D1sf0
  Processing URL 9/10: https://seekingalpha.com/article/4773017-april-2025-outlook-calamos-global-opportunities-fund


Error fetching article from https://seekingalpha.com/article/4773017-april-2025-outlook-calamos-global-opportunities-fund: Article `download()` failed with Status code 403 for url None on URL https://seekingalpha.com/article/4773017-april-2025-outlook-calamos-global-opportunities-fund
  Processing URL 10/10: https://www.chinadaily.com.cn/a/202504/16/WS67ff0e73a3104d9fd381f936.html
Waiting 5 seconds before next query...

Processing query 2/30: US presidential election updates
Found 10 URLs to process
  Processing URL 1/10: https://www.msn.com/en-gb/news/world/romanian-election-favourite-pledges-to-stop-potential-us-troop-withdrawal/ar-AA1CNC2p
  Processing URL 2/10: https://www.msn.com/en-us/news/world/ecuador-would-love-to-have-us-forces-helping-in-gang-crackdown-president-says/ar-AA1CZkn8
  Processing URL 3/10: https://www.msn.com/en-us/news/world/ecuador-s-president-daniel-noboa-declared-election-winner-as-rival-demands-recount/ar-AA1CQhcU
  Processing URL 4/10: https://www.msn.com/e

  Processing URL 6/10: https://www.fastcompany.com/91269023/artificial-intelligence-most-innovative-companies-2025
Error fetching article from https://www.fastcompany.com/91269023/artificial-intelligence-most-innovative-companies-2025: Article `download()` failed with Status code 403 for url None on URL https://www.fastcompany.com/91269023/artificial-intelligence-most-innovative-companies-2025
  Processing URL 7/10: https://www.fingerlakes1.com/2025/04/05/hochul-suny-ai-society-initiative-2025/
  Processing URL 8/10: https://www.govtech.com/policy/state-efficiency-bill-involving-ai-advances-in-north-carolina
  Processing URL 9/10: https://www.msn.com/en-us/news/technology/demis-hassabis-is-preparing-for-ai-s-endgame/ar-AA1D1VEL
  Processing URL 10/10: https://www.msn.com/en-us/news/other/nvidia-takes-5-5-billion-hit-as-us-clamps-down-on-its-china-chips/ar-AA1D1f3a
Waiting 5 seconds before next query...

Processing query 5/30: Major tech company earnings reports
Found 10 URLs to process

...

<html>
   <body>
      <h2>You might have been detected and blocked as a crawler bot!<h2>
      <h3>News Corp Australia uses software that manages crawler bot traffic on our websites. 


Error fetching article from https://www.theaustralian.com.au/business/dataroom/glencore-visit-spurs-rio-talk/news-story/07ccb47fa56c068e8a839546d3431583: Article `download()` failed with Status code 403 for url None on URL https://www.theaustralian.com.au/business/dataroom/glencore-visit-spurs-rio-talk/news-story/07ccb47fa56c068e8a839546d3431583
  Processing URL 9/10: https://www.foxbusiness.com/category/mergers-and-acquisitions
  Processing URL 10/10: https://www.benzinga.com/government/regulations/25/04/44653113/this-week-in-cannabis-trump-policy-updates-corporate-moves-regulatory-shifts-and-product-innovations

All article texts collected and saved to article_texts.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


File download initiated. Check your browser's downloads.

Summary statistics:
Total queries processed: 30
Total articles attempted: 300
Successfully parsed articles: 146 (48.7%)


In [ ]:
from duckduckgo_search import DDGS
import newspaper


query = "applications of blockchain in healthcare"
# results = list(DDGS().text(query, max_results=10))

for result in results:
    print(result['title'])
    print(result['body'])

Blockchain technology applications in healthcare: An overview
Blockchain is a relatively modern and emerging technology that has innovative applications during its successful healthcare implementation. Smooth, efficient data sharing and delivery across all the prominent network members and healthcare providers contribute to developing economical therapies and sophisticated treatments for many diseases.
Blockchain in Healthcare: 16 Real-World Examples - Built In
3 Ways Blockchain Will Change Healthcare | Video: The Medical Futurist Blockchain and Healthcare Data Security. In 2023, there were 725 data breaches of at least 500 patient records — the most ever reported to the Department of Health and Human Services Office for Civil Rights. Blockchain's ability to keep an incorruptible, decentralized and transparent log of all patient data makes it a ...
Frontiers | Exploring applications of blockchain in healthcare: road ...
Applications of blockchain in healthcare. Blockchain technology po

In [ ]:

article = newspaper.article('https://edition.cnn.com/2023/10/29/sport/nfl-week-8-how-to-watch-spt-intl/index.html')

print(article.authors)

['Hannah Brewitt']


In [ ]:
# news_result = DDGS().news(keywords="developments in climate change", region="us-en", safesearch="off", timelimit="m", max_results=10)
print(news_result)

[{'date': '2025-03-05T17:23:00+00:00', 'title': 'Opinion: Democracy and Climate Change', 'body': 'Ill-effects of growth model needs serious revisit to strike a balance between ecology, economy Climate change has emerged as both academic and policy relevance globally in the last two-and-a-half decades.', 'url': 'https://telanganatoday.com/opinion-democracy-and-climate-change', 'image': 'https://media.telanganatoday.com/wp-content/uploads/2025/03/Opinion.jpg', 'source': 'Telangana Today'}, {'date': '2025-03-06T04:42:00+00:00', 'title': 'Global South is driving climate agenda: Bhupender Yadav', 'body': 'The Global South is driving the climate agenda, and the world now looks at India as a leader, Union minister for environment, forest and climate change, Bhupender Yadav said on Wednesday.', 'url': 'https://www.msn.com/en-in/entertainment/southcinema/global-south-is-driving-climate-agenda-bhupender-yadav/ar-AA1AjA9j', 'image': 'https://www.livemint.com/lm-img/img/2025/03/05/1600x900/PTI02-1

In [ ]:
query = "applications of blockchain in healthcare"

# news_result = DDGS().news(
#     keywords=query,
#     region="us-en",
#     safesearch="off",
#     timelimit="m",
#     max_results=10)

# text_list = []

for result in news_result:
    try:
        article = newspaper.article(result['url'])
        article.download()
        article.parse()
        text_list.append({
            'title': result['title'],
            'body': result['body'],
            'url': result['url'],
            'full_text': article.text
        })
        print(article.title)
    except Exception as e:
        print(f"Skipping article due to error: {str(e)}")
        continue

Blockchain In Supply Chain: Real-World Applications In India
Navigating the Future of Technology With Broxus Blockchain Solutions for Governments and Enterprises


India's Crypto Start-up Ecosystem: Innovation In The Face Of Regulatory Uncertainty
Skipping article due to error: Article `download()` failed with Status code 403 for url None on URL https://www.tmcnet.com/usubmit/2025/03/06/10157071.htm
Blockchain’s Impact on Cryptocurrencies and Beyond
Ping An Tops Global Patentee Top 10 List, Leading in Fintech and Healthcare
Can AI Finally Solve the Healthcare Data Puzzle?
Top Blockchain Development Services for Web3 Solutions - Nadcab Labs
The Digital Transformation of Healthcare: AI, IoT, and Blockchain in Action
WISeKey WISeID Provides Healthcare Security with Decentralized Digital Identities


In [ ]:
!pip3 install sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.4 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=7cb5de9c167b757373d5c699ef101aa11f386ecf21dec52c30f77fa720a7faae
  Stored in directory: /root/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ac745418f864315a10278e6623643b37e5055f8dafc60e4269a6960db2943de7
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built breadability docopt


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from transformers import pipeline


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
def generate_combined_summary(text_list, method='abstractive'):
    # Concatenate all articles with titles
    combined_text = ""
    for item in text_list:
        combined_text += f"{item['title']}\n{item['full_text']}\n\n"

    if method == 'extractive':
        # Using extractive summarization
        parser = PlaintextParser.from_string(combined_text, Tokenizer("english"))
        summarizer = LexRankSummarizer()
        summary = summarizer(parser.document, sentences_count=10)
        return " ".join([str(sentence) for sentence in summary])

    elif method == 'abstractive':
        # Using transformers for abstractive summarization
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        # Process in chunks if the combined text is too long
        max_chunk_length = 1024  # Approximate token limit for BART
        chunks = [combined_text[i:i+max_chunk_length] for i in range(0, len(combined_text), max_chunk_length)]

        summaries = []
        for chunk in chunks:
            if len(chunk) > 100:  # Ensure chunk is long enough to summarize
                summary = summarizer(chunk, max_length=150, min_length=50, do_sample=False)
                summaries.append(summary[0]['summary_text'])

        return " ".join(summaries)

In [ ]:
# Generate a single summary from all articles
overall_summary = generate_combined_summary(text_list)
print("Abstractive Overall Summary:")
print(overall_summary)

In [ ]:
overall_summary_extractive = generate_combined_summary(text_list, 'extractive')
print("Extractive Overall Summary:")
print(overall_summary_extractive)

Extractive Overall Summary:
The Future of Blockchain in Supply Chain in India In Addition, Blockchain is revolutionizing many government services, from financing and supply chain management to healthcare, enabling decentralized transactions to reduce fraud and enhance data integrity. Therefore provides the integrity and security of the data with its transparency, making it an ideal application among digital currencies. The future of industries is going to be revolutionized by blockchain technology with its decentralized and secure nature beyond just digital currencies, building trust and transparency in the digital age. The Advancements Of Blockchain With Integration Of AI In The Future India's Crypto Start-up Ecosystem: Innovation In The Face Of Regulatory Uncertainty The innovation that has been triggered by the cryptos has had a chain reaction of innovation all over the world, and India is not an exception. The healthcare industry white paper points out that artificial intelligence 

In [ ]:
print(len(overall_summary_extractive))

1671


In [ ]:
def topic_based_multi_doc_summary(text_list, num_sentences=15):
    # Extract all sentences from all documents
    all_sentences = []
    for item in text_list:
        sentences = sent_tokenize(item['full_text'])
        all_sentences.extend(sentences)

    # Calculate TF-IDF
    # Changed stop_words to a list instead of a set
    stop_words = stopwords.words('english')
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(all_sentences)

    # Cluster sentences
    num_clusters = min(num_sentences, len(all_sentences))
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(tfidf_matrix)

    # Get closest sentences to cluster centers
    centers = kmeans.cluster_centers_
    ordered_sentences = []
    for i in range(num_clusters):
        # Find the sentence closest to this cluster center
        closest_idx = None
        closest_dist = float('inf')
        for j, sentence in enumerate(all_sentences):
            if kmeans.labels_[j] == i:
                # Convert tfidf_matrix[j] to dense array to match centers shape
                dist = ((tfidf_matrix[j].toarray() - centers[i]) ** 2).sum() ** 0.5
                if dist < closest_dist:
                    closest_dist = dist
                    closest_idx = j

        if closest_idx is not None:
            ordered_sentences.append((closest_idx, all_sentences[closest_idx]))

    # Sort by original order
    ordered_sentences.sort(key=lambda x: x[0])
    summary = " ".join([sentence for _, sentence in ordered_sentences])
    return summary

In [ ]:
# Generate topic-based summary
topic_summary = topic_based_multi_doc_summary(text_list)
print("Topic-based Summary:")
print(topic_summary)

Topic-based Summary:
The Future of Blockchain in Supply Chain in India

The application of blockchain technology in supply chain management will increase with growing awareness by businesses and industries of its significance. Such innovations will transform the dynamics of business operation and consumer engagement. With solutions like those offered by Broxus, the future of digital transformation will be robust, secure, scalable, and efficient. The future of industries is going to be revolutionized by blockchain technology with its decentralized and secure nature beyond just digital currencies, building trust and transparency in the digital age. The future of blockchain technology holds an immense potential with the integration of AI. The Prospect of Blockchain Technology

As blockchain technology continues to evolve, its potential applications are expected to grow exponentially. In the fintech sector, Ping An has 8,582 patents, accounting for more than 2% of patents of the entire ind

In [ ]:
print(len(topic_summary))

1887


In [ ]:
from collections import Counter
import re
import numpy as np

def weighted_multi_doc_summary(text_list, num_sentences=15):
    # Extract all text
    all_text = ""
    for item in text_list:
        all_text += item['full_text'] + " "

    # Find most common terms (potential key topics)
    words = re.findall(r'\w+', all_text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words and len(word) > 3]
    common_terms = [word for word, count in Counter(filtered_words).most_common(20)]

    # Score sentences based on common terms
    all_sentences = []
    for item in text_list:
        sentences = sent_tokenize(item['full_text'])
        for sentence in sentences:
            score = sum(1 for word in common_terms if word.lower() in sentence.lower())
            all_sentences.append((sentence, score))

    # Get top sentences
    all_sentences.sort(key=lambda x: x[1], reverse=True)
    top_sentences = [sentence for sentence, _ in all_sentences[:num_sentences]]

    # Order sentences as they appeared in original text
    sentence_positions = {}
    position = 0
    for item in text_list:
        sentences = sent_tokenize(item['full_text'])
        for sentence in sentences:
            if sentence in top_sentences:
                sentence_positions[sentence] = position
            position += 1

    ordered_top_sentences = sorted(top_sentences, key=lambda s: sentence_positions.get(s, float('inf')))
    summary = " ".join(ordered_top_sentences)
    return summary

In [ ]:
# Generate weighted summary
weighted_summary = weighted_multi_doc_summary(text_list)
print("Weighted Summary:")
print(weighted_summary)

Weighted Summary:
Understanding Supply Chain Management in Blockchain

Blockchain is a decentralized, immutable digital ledger that guarantees secure, transparent transactions. The Future of Blockchain in Supply Chain in India

The application of blockchain technology in supply chain management will increase with growing awareness by businesses and industries of its significance. In Addition, Blockchain is revolutionizing many government services, from financing and supply chain management to healthcare, enabling decentralized transactions to reduce fraud and enhance data integrity. For enterprises, their solutions offer real-world asset tokenization, supply chain automation, and decentralization of financial services to improve operational efficiency and data security. Understanding The Security and Basics of Blockchain Technology

Blockchain is a technology with a decentralized distributed ledger that allows P2P transactions to facilitate the safest direct exchange between two partie

In [ ]:
print(len(weighted_summary))

3567


In [ ]:
print(len(text_list))
print(text_list[0])

9
{'title': 'Blockchain In Supply Chain: Real-World Applications In India', 'body': 'Blockchain makes transformation in supply chain management offering clarity, efficiency and security. It has real-life applications in India which can tremendously range from tracking food safety and preventing counterfeiting activities and finally to cost reduction.', 'url': 'https://www.outlookindia.com/xhub/blockchain-insights/blockchain-in-supply-chain-real-world-applications-in-india', 'full_text': "Understanding Supply Chain Management in Blockchain\n\nBlockchain is a decentralized, immutable digital ledger that guarantees secure, transparent transactions. Unlike traditional databases that consolidate their control in one place, blockchain guarantees that data is always distributed across many nodes, making it nearly impossible to alter or manipulate. This decentralized aspect fortifies the trust and security of supply chains, as every transaction must be verified by all network participants prio

In [ ]:
import time

# Function to get news with proper rate limiting
def get_rate_limited_news(keywords, delay_seconds=10, max_results=10, **kwargs):
    ddgs = DDGS()
    results = []

    # Set a smaller batch size to get fewer results per API call
    batch_size = 1
    remaining = max_results

    while remaining > 0:
        # Calculate how many results to request in this batch
        current_batch = min(batch_size, remaining)

        # Make the API call for just this batch
        batch_generator = ddgs.news(
            keywords=keywords,
            region="us-en",
            safesearch="off",
            timelimit="m",
            max_results=current_batch,
            **kwargs
        )

        # Process results from this batch
        batch_results = list(batch_generator)
        results.extend(batch_results)

        print(f"Fetched {len(batch_results)} articles (total: {len(results)}/{max_results})")

        # Update remaining count
        remaining -= len(batch_results)

        # Don't sleep after the last batch
        if remaining > 0:
            print(f"Waiting {delay_seconds} seconds before next request...")
            time.sleep(delay_seconds)

    return results



In [ ]:
# Use the rate-limited function
climate_news = get_rate_limited_news(
    keywords="developments in climate change",
    delay_seconds=10,
    max_results=10
)

# Now you have your results with proper rate limiting
print(f"\nTotal articles fetched: {len(climate_news)}")

Fetched 1 articles (total: 1/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 2/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 3/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 4/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 5/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 6/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 7/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 8/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 9/10)
Waiting 10 seconds before next request...
Fetched 1 articles (total: 10/10)

Total articles fetched: 10


In [ ]:
print(climate_news)

[{'date': '2025-03-05T17:23:00+00:00', 'title': 'Opinion: Democracy and Climate Change', 'body': 'Ill-effects of growth model needs serious revisit to strike a balance between ecology, economy Climate change has emerged as both academic and policy relevance globally in the last two-and-a-half decades.', 'url': 'https://telanganatoday.com/opinion-democracy-and-climate-change', 'image': 'https://media.telanganatoday.com/wp-content/uploads/2025/03/Opinion.jpg', 'source': 'Telangana Today'}, {'date': '2025-03-06T16:09:39+00:00', 'title': 'World Must Prepare to Fight Climate Change Without US, UK Says', 'body': "The world needs to prepare to fight global warming without the help of the US as the Trump administration pulls billions of dollars pledged to initiatives funding the green transition, the UK's climate envoy said.", 'url': 'https://www.msn.com/en-us/money/other/world-must-prepare-to-fight-climate-change-without-us-uk-says/ar-AA1Aoc0Q', 'image': 'https://img-s-msn-com.akamaized.net/t

In [ ]:
climate_news_full_articles = []

for result in climate_news:
    try:
        article = newspaper.article(result['url'])
        article.download()
        article.parse()
        climate_news_full_articles.append({
            'title': result['title'],
            'body': result['body'],
            'url': result['url'],
            'full_text': article.text
        })
    except Exception as e:
        print(f"Skipping article due to error: {str(e)}")
        continue

In [ ]:
print(len(climate_news_full_articles))

10


In [ ]:
# Generate weighted summary
weighted_summary_climate = weighted_multi_doc_summary(climate_news_full_articles)
print("Weighted Summary Climate:")
print(weighted_summary_climate)

Weighted Summary Climate:
Ill-effects of growth model needs serious revisit to strike a balance between ecology, economy

By Nayakara Veeresha

Climate change has emerged as both academic and policy relevance globally in the last two-and-a-half decades. Ill-effects of growth model needs serious revisit to strike a balance between ecology, economy

By Nayakara Veeresha

Climate change has emerged as both academic and policy relevance globally in the last two-and-a-half decades. Ill-effects of growth model needs serious revisit to strike a balance between ecology, economy

By Nayakara Veeresha

Climate change has emerged as both academic and policy relevance globally in the last two-and-a-half decades. Ill-effects of growth model needs serious revisit to strike a balance between ecology, economy

By Nayakara Veeresha

Climate change has emerged as both academic and policy relevance globally in the last two-and-a-half decades. Despite this marked shift, the response to climate change-rela

In [ ]:
# Generate topic-based summary
topic_summary_climate = topic_based_multi_doc_summary(climate_news_full_articles)
print("Topic-based Summary for Climate:")
print(topic_summary_climate)

Topic-based Summary for Climate:
Despite this marked shift, the response to climate change-related issues and challenges is not significant across nations, especially among the rich countries. However, concrete actions are missing. The Covid pandemic has brought out the limitations of macroeconomic management. The structural and institutional constraints need an alternative economic governance regime. The buzzword of the circular economy is part and parcel of the global discourse on climate change. The development model pursued has focused on the linear growth pattern by distancing from the cultural roots of circular economic growth nested in ecology and sustainability. Forces of liberalisation, privatisation and globalisation are exerting pressure on state governments to ease the process of forest clearance for mining and other natural resources

The ill-effects of the ongoing economic growth model and development path both need a serious revisit to strike a balance between the ecolog

In [ ]:
print(len(topic_summary_climate))

2324


In [ ]:
# Generate a single summary from all articles
overall_summary = generate_combined_summary(climate_news_full_articles)
print("Abstractive Overall Summary:")
print(overall_summary)

Device set to use cpu


Abstractive Overall Summary:
Ill-effects of growth model needs serious revisit to strike a balance between ecology, economy. Climate change has emerged as both academic and policy relevance globally. Despite this marked shift, the response to climate change-related issues and challenges is not significant across nations, especially among the rich countries. The buzzword of the circular economy is part and parcel of the global discourse on climate change. The development model pursued has focused on the linear growth pattern by distancing from the cultural roots of circular economic growth nested in ecology and sustainability. The ill-effects of the ongoing economic growth model and development path both need a serious revisit. Growth and development attained on these premises are unsustainable. Ecology and environment must be given the highest priority in the policy-making process. The economy nested within the ecology and the growth embedded in the sustainability component will addres

In [ ]:
print(len(overall_summary))

8624


In [ ]:
def compress_summary(summary, max_length=800):
    sentences = sent_tokenize(summary)

    # If already short enough, return as is
    if len(summary) <= max_length:
        return summary

    # Calculate importance score for each sentence using key phrase presence
    key_phrases = ["climate change", "global warming", "carbon emissions",
                  "renewable energy", "policy", "agreement", "temperature"]

    scored_sentences = []
    for sentence in sentences:
        # Score based on presence of key phrases and sentence position
        score = sum(2 if phrase in sentence.lower() else 0 for phrase in key_phrases)
        scored_sentences.append((sentence, score))

    # Sort by score and select top sentences
    scored_sentences.sort(key=lambda x: x[1], reverse=True)

    # Keep selecting sentences until we reach the length limit
    compressed = []
    current_length = 0
    for sentence, _ in scored_sentences:
        if current_length + len(sentence) <= max_length:
            compressed.append(sentence)
            current_length += len(sentence) + 1  # +1 for the space
        else:
            break

    return " ".join(compressed)

# Apply compression
compressed_topic_summary = compress_summary(overall_summary, max_length=600)
print(f"Original length: {len(overall_summary)}")
print(f"Compressed length: {len(compressed_topic_summary)}")

Original length: 8624
Compressed length: 585


In [ ]:
print(compressed_topic_summary)

Climate change has emerged as both academic and policy relevance globally. Despite this marked shift, the response to climate change-related issues and challenges is not significant across nations, especially among the rich countries. The buzzword of the circular economy is part and parcel of the global discourse on climate change. Ecology and environment must be given the highest priority in the policy-making process. The political economy of the carbon emissions and its trade between the Global South and Northern countries is one of the major factors for the lack of consensus.


In [ ]:
def remove_redundancy(summary):
    sentences = sent_tokenize(summary)
    unique_sentences = []
    content_seen = set()

    for sentence in sentences:
        # Create a simplified representation by keeping only important words
        important_words = [w.lower() for w in word_tokenize(sentence)
                          if w.lower() not in stopwords.words('english') and len(w) > 3]

        # Skip sentences that are too similar to ones we've already kept
        sentence_content = " ".join(sorted(important_words))
        similarity_found = False

        for existing in content_seen:
            # Calculate Jaccard similarity
            common = len(set(sentence_content.split()) & set(existing.split()))
            total = len(set(sentence_content.split()) | set(existing.split()))
            if total > 0 and common/total > 0.6:  # More than 60% similar
                similarity_found = True
                break

        if not similarity_found:
            unique_sentences.append(sentence)
            content_seen.add(sentence_content)

    return " ".join(unique_sentences)

# Apply redundancy removal
non_redundant_summary = remove_redundancy(overall_summary)
print(f"After redundancy removal: {len(non_redundant_summary)}")

After redundancy removal: 4461


In [ ]:
def abstractive_compress(text, max_length=150, min_length=50):
    # Initialize summarizer
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    # If text is too long, break it into chunks and summarize each
    if len(text) > 1024:
        # Split into chunks of about 1000 characters
        chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
        summaries = []

        for chunk in chunks:
            if len(chunk) > 100:  # Only summarize substantial chunks
                summary = summarizer(chunk, max_length=max_length//len(chunks),
                                    min_length=min_length//len(chunks))
                summaries.append(summary[0]['summary_text'])

        return " ".join(summaries)
    else:
        # Summarize directly
        summary = summarizer(text, max_length=max_length, min_length=min_length)
        return summary[0]['summary_text']

# Apply abstractive compression to further reduce size
final_summary = abstractive_compress(topic_summary_climate, max_length=300, min_length=200)
print(f"Final summary length: {len(final_summary)}")

Device set to use cpu
Your max_length is set to 100, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


Final summary length: 1206


In [ ]:
print(final_summary)

The buzzword of the circular economy is part and parcel of the global discourse on climate change. Despite this marked shift, the response to climate change-related issues and challenges is not significant across nations. Forces of liberalisation, privatisation and globalisation are exerting pressure on state governments to ease the process of forest clearance for mining. The political economy of the carbon emissions and its trade between the Global South and Northern countries is one of the major factors for the lack of consensus. It also highlights the inadequacy of the market structure to facilitate the climatic action plan among the developed and developing nations. The emergence and commitment of the European Union (EU) in the form of Green Deal to an extent is the silver lining in achieving the target of SDGs by 2030, especially Goal 13. 10 out of 13 indicators still in Tier III. This needs revisiting the economic growth model by improving institutional capacities of the state al

In [ ]:
final_summary_2 = abstractive_compress(final_summary, max_length=30, min_length=10)
print(f"Final summary length: {len(final_summary_2)}")

Device set to use cpu


Final summary length: 142


In [ ]:
print(final_summary_2)

The response to climate change-related issues and challenges is not The UN Secretary-General says democracy is critical in developing globally


In [2]:
import requests
import json

target_summaries_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/new_target.json"

# Fetch the JSON data from the URL
response = requests.get(target_summaries_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
target_data = json.loads(response.text)

# Create a dictionary mapping query to summary
target_summaries = {item['query']: item['summary'] for item in target_data}

# Display information about the loaded data
print(f"Loaded {len(target_summaries)} target summaries")
print("Sample target summary:")
first_key = list(target_summaries.keys())[0]
print(f"Query: {first_key}")
print(f"Summary: {target_summaries[first_key][:200]}...")

Loaded 30 target summaries
Sample target summary:
Query: Global economic outlook
Summary: The global economic outlook for 2025 is marked by uncertainty, with trade tensions and tariffs weighing heavily on investor sentiment. Surveys indicate the most bearish outlook in decades, largely due...


In [3]:
news_urls_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/news_urls.json"
response = requests.get(news_urls_json_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
news_urls = json.loads(response.text)
print(f"Loaded URLs for {len(news_urls)} topics")
print("Sample topic with URLs:")
first_key = list(news_urls.keys())[0]
print(f"Topic: {first_key}")
print(f"URLs: {news_urls[first_key]}")

Loaded URLs for 30 topics
Sample topic with URLs:
Topic: Global economic outlook
URLs: ['https://www.msn.com/en-ca/money/topstories/global-economic-growth-could-slow-to-23-due-to-trade-tensions-says-un-trade-agency/ar-AA1D1z6Q', 'https://seekingalpha.com/article/4775157-how-tariffs-geopolitics-shaping-2025-global-economic-outlook', 'https://www.cnet.com/personal-finance/refinance-rates-tick-higher-mortgage-refinance-rates-on-april-11-2025/', 'https://www.businessinsider.com/us-economy-stock-market-outlook-tariffs-trump-trade-war-bofa-2025-4', 'https://www.msn.com/en-us/money/markets/economic-outlook-dives-just-three-months-into-trump-s-term/ar-AA1COpd8', 'https://www.msn.com/en-us/money/markets/u-k-economy-shows-signs-of-strength-but-tariffs-darken-outlook/ar-AA1CIv5r', 'https://www.marketwatch.com/livecoverage/stock-market-today-dow-s-p-and-nasdaq-eye-steady-start-as-tariff-angst-eases-bofa-and-citi-earnings/card/iea-cuts-2025-global-demand-growth-forecast-on-deteriorating-outlook-for

In [5]:
article_texts_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/article_texts.json"
response = requests.get(article_texts_json_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
article_texts = json.loads(response.text)
print(f"Loaded article texts for {len(article_texts)} topics")
topics_with_articles = sum(1 for articles in article_texts.values() if articles)
print(f"Number of topics with at least one article: {topics_with_articles}")

Loaded article texts for 30 topics
Number of topics with at least one article: 30


In [7]:
import nltk
# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

True

In [8]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

# Function to extract first 150 words from text
def extract_first_150_words(text):
    """Extract the first 150 words from an article."""
    if not text or not isinstance(text, str) or not text.strip():
        return ""
    words = word_tokenize(text)
    first_150 = ' '.join(words[:150])
    return first_150

# Generate summaries from the first 150 words of the first article for each query
generated_summaries = {}
for query, articles in article_texts.items():
    if articles and len(articles) > 0:
        # Find the first article with actual text
        article_text = ""
        for article in articles:
            if isinstance(article, dict) and 'text' in article and article['text']:
                article_text = article['text']
                break

        if article_text:
            generated_summaries[query] = extract_first_150_words(article_text)
        else:
            print(f"No valid text found for query: {query}")
    else:
        print(f"No articles found for query: {query}")

print(f"Generated {len(generated_summaries)} summaries")
# Show an example generated summary
if generated_summaries:
    example_query = list(generated_summaries.keys())[0]
    print(f"Example generated summary for '{example_query}':\n{generated_summaries[example_query][:200]}...")

Generated 30 summaries
Example generated summary for 'Global economic outlook':
Mortgage refinance rates have been moderating in response to rising layoffs and worries about a potential economic downturn . Despite the recent dip in rates , experts don ’ t expect another refinanci...
